In [4]:
import sqlite3
import pandas as pd

# Connect to SQLite database (or create it)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE Customers (
    CustomerID INTEGER PRIMARY KEY,
    FirstName TEXT,
    LastName TEXT,
    Email TEXT,
    DateOfBirth DATE
);
''')

cursor.execute('''
CREATE TABLE Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT,
    Price REAL
);
''')

cursor.execute('''
CREATE TABLE Orders (
    OrderID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    OrderDate DATE,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
''')

cursor.execute('''
CREATE TABLE OrderItems (
    OrderItemID INTEGER PRIMARY KEY,
    OrderID INTEGER,
    ProductID INTEGER,
    Quantity INTEGER,
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);
''')

# Insert sample data
cursor.execute('''
INSERT INTO Customers (CustomerID, FirstName, LastName, Email, DateOfBirth)
VALUES
    (1, 'John', 'Doe', 'john.doe@example.com', '1985-01-15'),
    (2, 'Jane', 'Smith', 'jane.smith@example.com', '1990-06-20');
''')

cursor.execute('''
INSERT INTO Products (ProductID, ProductName, Price)
VALUES
    (1, 'Laptop', 1000),
    (2, 'Smartphone', 600),
    (3, 'Headphones', 100);
''')

cursor.execute('''
INSERT INTO Orders (OrderID, CustomerID, OrderDate)
VALUES
    (1, 1, '2023-01-10'),
    (2, 2, '2023-01-12');
''')

cursor.execute('''
INSERT INTO OrderItems (OrderItemID, OrderID, ProductID, Quantity)
VALUES
    (1, 1, 1, 1),
    (2, 1, 3, 2),
    (3, 2, 2, 1),
    (4, 2, 3, 1);
''')

# Commit the changes
conn.commit()

# Function to execute and display queries
def execute_query(query):
    result = pd.read_sql_query(query, conn)
    print(result)

# 1. List all customers
query1 = "SELECT * FROM Customers;"
execute_query(query1)

print()
print("--------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------")
print()

# 2. Find all orders placed in January 2023
query2 = "SELECT * FROM Orders WHERE OrderDate BETWEEN '2023-01-01' AND '2023-01-31';"
execute_query(query2)

print()
print("--------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------")
print()

# 3. Get the details of each order, including the customer name and email
query3 = '''
SELECT o.OrderID, c.FirstName, c.LastName, c.Email, o.OrderDate
FROM Orders o
JOIN Customers c ON o.CustomerID = c.CustomerID;
'''
execute_query(query3)

print()
print("--------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------")
print()

# 4. List the products purchased in a specific order (e.g., OrderID = 1)
query4 = '''
SELECT p.ProductName, oi.Quantity
FROM OrderItems oi
JOIN Products p ON oi.ProductID = p.ProductID
WHERE oi.OrderID = 1;
'''
execute_query(query4)

print()
print("--------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------")
print()

# 5. Calculate the total amount spent by each customer
query5 = '''
SELECT c.CustomerID, c.FirstName, c.LastName, SUM(p.Price * oi.Quantity) AS TotalSpent
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN OrderItems oi ON o.OrderID = oi.OrderID
JOIN Products p ON oi.ProductID = p.ProductID
GROUP BY c.CustomerID, c.FirstName, c.LastName;
'''
execute_query(query5)

print()
print("--------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------")
print()

# 6. Find the most popular product (the one that has been ordered the most)
query6 = '''
SELECT p.ProductID, p.ProductName, SUM(oi.Quantity) AS TotalOrdered
FROM OrderItems oi
JOIN Products p ON oi.ProductID = p.ProductID
GROUP BY p.ProductID, p.ProductName
ORDER BY TotalOrdered DESC
LIMIT 1;
'''
execute_query(query6)

print()
print("--------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------")
print()

# 7. Get the total number of orders and the total sales amount for each month in 2023
query7 = '''
SELECT 
    strftime('%Y-%m', o.OrderDate) AS Month, 
    COUNT(o.OrderID) AS TotalOrders, 
    SUM(p.Price * oi.Quantity) AS TotalSales
FROM Orders o
JOIN OrderItems oi ON o.OrderID = oi.OrderID
JOIN Products p ON oi.ProductID = p.ProductID
WHERE strftime('%Y', o.OrderDate) = '2023'
GROUP BY strftime('%Y-%m', o.OrderDate);
'''
execute_query(query7)

print()
print("--------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------")
print()

# 8. Find customers who have spent more than $1000
query8 = '''
SELECT c.CustomerID, c.FirstName, c.LastName, SUM(p.Price * oi.Quantity) AS TotalSpent
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN OrderItems oi ON o.OrderID = oi.OrderID
JOIN Products p ON oi.ProductID = p.ProductID
GROUP BY c.CustomerID, c.FirstName, c.LastName
HAVING TotalSpent > 1000;
'''
execute_query(query8)

print()
print("--------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------")
print()

# Close the connection
conn.close()


   CustomerID FirstName LastName                   Email DateOfBirth
0           1      John      Doe    john.doe@example.com  1985-01-15
1           2      Jane    Smith  jane.smith@example.com  1990-06-20

--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------

   OrderID  CustomerID   OrderDate
0        1           1  2023-01-10
1        2           2  2023-01-12

--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------

   OrderID FirstName LastName                   Email   OrderDate
0        1      John      Doe    john.doe@example.com  2023-01-10
1        2      Jane    Smith  jane.smith@example.com  2023-01-12

--------------------------------------------------------------------------------------
--------------------------------------------------